In [1]:
import pickle
# 导入向量化后的数据集
np_vectorized_tfidf = pickle.load(open('model/vectorized_generated_corpus_statistical_normalized.pkl', 'rb'))

In [2]:
# 读入正确标签（Labels）
labels = []
# 读入数据集
with open('sixTypes-GeneratedLabels.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        labels.append(line.strip('\n'))

In [3]:
# 利用 RandomizedSearchCV 搜索不同的 维度 和 gamma 值来找到最优解（利用 V-measure 评估）

# Import required libraries
from sklearn.model_selection import RandomizedSearchCV
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import v_measure_score

# Define the parameter values that want to search
param_grid = {
    'lle__n_components': range(2, 11),
    'lle__n_neighbors': range(2, 31),
    'kmeans__n_clusters': range(6, 7)
}

# Define a scoring function that takes in y_true and y_pred as arguments and returns the v_measure_score
def v_measure_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return v_measure_score(y_true, y_pred)

# Create a randomize search object
lle = LocallyLinearEmbedding()
kmeans = KMeans(init='k-means++', random_state=9)
model = Pipeline([('lle', lle), ('kmeans', kmeans)])
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=5, random_state=9, scoring=v_measure_scorer, verbose=3)

# Fit the random search object to the data
random_search.fit(np_vectorized_tfidf, labels)

# Get the best estimator
best_estimator = random_search.best_estimator_
print('best_estimator', best_estimator)

# Use the best estimator to make predictions
y_pred = best_estimator.predict(np_vectorized_tfidf)

# Use V-measure to evaluate the clustering results
score = v_measure_score(labels, y_pred)
print('V-measure:', score)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END kmeans__n_clusters=6, lle__n_components=3, lle__n_neighbors=27;, score=0.133 total time=   0.5s
[CV 2/5] END kmeans__n_clusters=6, lle__n_components=3, lle__n_neighbors=27;, score=0.022 total time=   0.3s
[CV 3/5] END kmeans__n_clusters=6, lle__n_components=3, lle__n_neighbors=27;, score=0.137 total time=   0.3s
[CV 4/5] END kmeans__n_clusters=6, lle__n_components=3, lle__n_neighbors=27;, score=0.016 total time=   0.3s
[CV 5/5] END kmeans__n_clusters=6, lle__n_components=3, lle__n_neighbors=27;, score=0.007 total time=   0.3s
[CV 1/5] END kmeans__n_clusters=6, lle__n_components=7, lle__n_neighbors=23;, score=0.028 total time=   0.3s
[CV 2/5] END kmeans__n_clusters=6, lle__n_components=7, lle__n_neighbors=23;, score=0.021 total time=   0.3s
[CV 3/5] END kmeans__n_clusters=6, lle__n_components=7, lle__n_neighbors=23;, score=0.059 total time=   0.4s
[CV 4/5] END kmeans__n_clusters=6, lle__n_components=7, lle__n_neig

In [4]:
from sklearn.manifold import LocallyLinearEmbedding
# 选用最优解维数用 LLE 降维
lle = LocallyLinearEmbedding(n_components=7, n_neighbors=9)
decomposition_data = lle.fit_transform(np_vectorized_tfidf)

In [5]:
# 选用 k 值做 K-means 算法聚类
y_pred = KMeans(n_clusters=6, init='k-means++', random_state=9).fit_predict(decomposition_data)

In [6]:
print(y_pred[:10])

[3 3 3 3 1 3 3 3 3 3]


In [7]:
from sklearn import metrics

In [8]:
print('%.2f' % metrics.calinski_harabasz_score(decomposition_data, y_pred))
km = KMeans(n_clusters=6, random_state=9)
y_pred = km.fit_predict(decomposition_data)
print('%.4f' % metrics.silhouette_score(decomposition_data, km.labels_, metric='euclidean'))
print('%.4f' % metrics.homogeneity_score(labels, y_pred))
print('%.4f' % metrics.completeness_score(labels, y_pred))
print('%.4f' % metrics.v_measure_score(labels, y_pred))
print('%.4f' % metrics.adjusted_rand_score(labels, y_pred))
print('%.4f' % metrics.adjusted_mutual_info_score(labels, y_pred))
print('%.4f' % metrics.fowlkes_mallows_score(labels, y_pred))

297.01
0.3781
0.5588
0.6803
0.6136
0.4200
0.6110
0.5613


In [80]:
# 利用 Calinski Harabasz Score 评估当前 K-Means 聚类效果（利用 TFIDF 表征）
metrics.calinski_harabasz_score(decomposition_data, y_pred)

8358.12173286958

In [81]:
# 利用 Silhouette Score（轮廓系数）评估当前 K-Means 聚类效果（利用 TFIDF 表征）
km = KMeans(n_clusters=6, random_state=9)
y_pred = km.fit_predict(decomposition_data)
metrics.silhouette_score(decomposition_data, km.labels_, metric='euclidean')

0.7227561603932742

In [82]:
# 读入正确标签（Labels）
labels = []
# 读入数据集
with open('sixTypes-GeneratedLabels.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        labels.append(line.strip('\n'))

In [83]:
labels[:10]

['CP', 'CP', 'CP', 'CP', 'CP', 'CP', 'CP', 'CP', 'CP', 'CP']

In [84]:
# 利用 Homogeneity 评估当前 K-Means 聚类效果（利用 TFIDF 表征）
metrics.homogeneity_score(labels, y_pred)

0.682122778322645

In [85]:
# 利用 Completeness 评估当前 K-Means 聚类效果（利用 TFIDF 表征）
metrics.completeness_score(labels, y_pred)

0.7476119515684342

In [86]:
# 利用 V-measure 评估当前 K-Means 聚类效果（利用 TFIDF 表征）
metrics.v_measure_score(labels, y_pred)

0.7133674951715347

In [87]:
# 利用 Adjusted Rand Index（调整兰德系数）评估当前 K-Means 聚类效果（利用 TFIDF 表征）
metrics.adjusted_rand_score(labels, y_pred)

0.5925578028425994

In [88]:
# 利用 Adjusted Mutual Information Score（调整互信息）评估当前 K-Means 聚类效果（利用 TFIDF 表征）
metrics.adjusted_mutual_info_score(labels, y_pred)

0.7115960614082019

In [89]:
# 利用 Fowlkes-Mallows scores 评估当前 K-Means 聚类效果（利用 TFIDF 表征）
metrics.fowlkes_mallows_score(labels, y_pred)

0.6776284744379846